## Alternative Process Plan Scheduling

[Slides from the video](https://icaps17.icaps-conference.org/tutorials/T3-Introduction-to-CP-Optimizer-for-Scheduling.pdf)

Slide 65 -- Structural constraints

**Span constraint - (AND node)**

span(x,[y1,...,yn])

if x is present it spands all rpesent intervals from {y1,...yn}
that is. at least one of yi supports the start (resp. end) of x

if x is absent, then all yi are absent to

**Alternative constraint - (OR node)**

alterantive(x, [y1,...,yn])
if x is present, then **exactly one** of the {y1,..,yn} is present and synchronized with x (sa,e start and end value)

if x is absent then all yi are absent too

alternative(x, [y1,...,yn],k) k:integer expression is x is present then exaclt k of the {y1,...,yn} are present and synchronized with x (same start and end value) if x is absent, then all yi are absent too

combine with presenceOf(task[i]) => presenceOf(task[j])

Sequence constraints

Sequencing constraints are unary constraints on a sequence variable p
- first(p,x) : if x is present it is the first one in the permutation
- last(p,x) : if x is present it is the last one in the permutation
- prev(p,x,y) : if both x and y are present, x appears right before y in the permutation
- before(p,x,y) : if both x and y are present, x appears somewhere before y in the permutation